In [1]:
from RandomForest import spark, process_dataframe, sc
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

23/05/09 19:48:59 WARN Utils: Your hostname, codespaces-662bcc resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/05/09 19:48:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/09 19:49:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Training

In [2]:
df = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("../../DataFiles/airline-train.csv")

df, string_indexers, bucketizers = process_dataframe(df)

In [3]:
LABEL_COL = "satisfaction_index"
FEATURES_COL = list(set(df.columns) - set([LABEL_COL]))
print("Label used: ", LABEL_COL)
print("Features used: ", FEATURES_COL)

Label used:  satisfaction_index
Features used:  ['Arrival Delay in Minutes_bucket', 'Gate location', 'Inflight wifi service', 'Online boarding', 'Type of Travel_index', 'Inflight entertainment', 'Gender_index', 'Seat comfort', 'Class_index', 'Customer Type_index', 'Flight Distance_bucket', 'Age_bucket', 'On-board service', 'Leg room service', 'Ease of Online booking', 'Departure Delay in Minutes_bucket', 'Food and drink', 'Baggage handling', 'Inflight service', 'Cleanliness', 'Departure/Arrival time convenient', 'Checkin service']


In [4]:
assembler = VectorAssembler(inputCols=FEATURES_COL, outputCol="features")
df = assembler.transform(df)

In [14]:
rf = RandomForestClassifier(labelCol=LABEL_COL, featuresCol="features", numTrees=50)
model = rf.fit(df)
# Training accuracy
accuracy = model.transform(df).select("satisfaction_index", "prediction").rdd.map(lambda x: 1.0 if x[0] == x[1] else 0.0).mean()
print("Training Accuracy: ", accuracy)

Training Accuracy:  0.9238179817363943


## Validation

In [15]:
df_val = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("../../DataFiles/airline-val.csv")

df_val, _, _ = process_dataframe(df_val, string_indexers, bucketizers)
assembler = VectorAssembler(inputCols=FEATURES_COL, outputCol="features")
df_val = assembler.transform(df_val)

In [16]:
# Validation accuracy
accuracy = model.transform(df_val).select("satisfaction_index", "prediction").rdd.map(lambda x: 1.0 if x[0] == x[1] else 0.0).mean()
print("Validation Accuracy: ", accuracy)

Validation Accuracy:  0.9243038659097063
